### Sustaining POWER
    Projected Hackathon: Arctic League of Code (Devpost)
    Author: Melvin T (https://devpost.com/melvin-t)

In [1]:
# File: models.py

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor

def train_model(X, y, model_type):
    if model_type == "linear":
        model = LinearRegression()
        model.fit(X, y)
        return model, None

    if model_type == "poly":
        poly = PolynomialFeatures(degree=3)
        Xp = poly.fit_transform(X)
        model = LinearRegression().fit(Xp, y)
        return model, poly

    if model_type == "rf":
        model = RandomForestRegressor(n_estimators=200)
        model.fit(X, y)
        return model, None

In [9]:
# File: analytics.py

import polars as pl
import numpy as np

def analyze(entity, model_type, future_year):
    df = pl.read_csv("data.csv", infer_schema_length=10000, try_parse_dates=True)
    c = df.filter(pl.col("Entity") == entity)

    X = c["Year"].to_numpy().reshape(-1,1)
    y = c["Primary energy consumption per capita (kWh/person)"].to_numpy()

    model, transformer = train_model(X, y, model_type)

    years = np.arange(2000, future_year + 1).reshape(-1,1)
    Xf = transformer.transform(years) if transformer else years
    preds = model.predict(Xf)

    energy_mix = {
        "Fossil": float(c["Electricity from fossil fuels (TWh)"].mean()),
        "Renewables": float(c["Electricity from renewables (TWh)"].mean()),
        "Nuclear": float(c["Electricity from nuclear (TWh)"].mean())
    }

    return {
        "years": list(range(2000, future_year + 1)),
        "predictions": preds.tolist(),
        "energy_mix": energy_mix,
        "location": {
            "entity": entity,
            "lat": float(c["Latitude"][0]),
            "lon": float(c["Longitude"][0])
        }
    }

In [67]:
# File: app.py

# from flask import Flask, request, jsonify
# from analytics import analyze

# app = Flask(__name__)

# @app.route("/analyze", methods=["POST"])
# def run():
#     data = request.json
#     result = analyze(data["entity"], data["model"], int(data["year"]))

#     result["insight"] = (
#         f"Projected energy demand rises steadily, indicating "
#         f"the need for accelerated renewable investment to meet SDG 7 targets."
#     )

#     return jsonify(result)

# if __name__ == "__main__":
#     app.run(debug=True)

In [102]:
# As Jupyter Notebook does not support JSON, we use the following as replacement:

if __name__ == "__main__":

    def output(model_choice):
        result = analyze(entity_name, model_choice, future_year)
        print(f"\n-------------------- '{model_choice}' Model - Predictive Analysis Outcome --------------------")
        print("Accurate Location:", result["location"])
        print("Projected Primary Energy Consumption per capita (kWh/person):", result["predictions"][-1])
        print("Energy Mix:", result["energy_mix"])
        print(f"\nIn the target year ({future_year}), the average person in {entity_name} is predicted to consume about {result["predictions"][-1]:.2f}kWh of energy, as predicted by the {model_choice} model.\n")
        view_check = input("To view the full prediction, type 'FP' (case-sensitive): ")
        if view_check == "FP":
            print("\n", result)

    entity_name = input("Country Name: ")
    future_year = int(input("Year (To Predict): "))
    model_amt = input("Use all 3 models (linear, poly, rf)? (y/n): ")
    print("\nCount of Years (incl. 2020):", len(result["years"]))
    if (model_amt == "Y") or (model_amt == "y"):
        for i in ["linear", "poly", "rf"]:
            output(i)
    else:
        model_choice = input("Model Choice: ")
        output(model_choice)

Country Name:  United States
Year (To Predict):  3020
Use all 3 models (linear, poly, rf)? (y/n):  y



Count of Years (incl. 2020): 31

-------------------- 'linear' Model - Predictive Analysis Outcome --------------------
Accurate Location: {'entity': 'United States', 'lat': 37.09024, 'lon': -95.712891}
Projected Primary Energy Consumption per capita (kWh/person): -818563.7784935064
Energy Mix: {'Fossil': 2745.3333333333335, 'Renewables': 487.33809523809515, 'Nuclear': 790.6004761904763}

In the target year (3020), the average person in United States is predicted to consume about -818563.78kWh of energy, as predicted by the linear model.



To view the full prediction, type 'FP' (case-sensitive):  



-------------------- 'poly' Model - Predictive Analysis Outcome --------------------
Accurate Location: {'entity': 'United States', 'lat': 37.09024, 'lon': -95.712891}
Projected Primary Energy Consumption per capita (kWh/person): 2668713982.0303497
Energy Mix: {'Fossil': 2745.3333333333335, 'Renewables': 487.33809523809515, 'Nuclear': 790.6004761904763}

In the target year (3020), the average person in United States is predicted to consume about 2668713982.03kWh of energy, as predicted by the poly model.



To view the full prediction, type 'FP' (case-sensitive):  



-------------------- 'rf' Model - Predictive Analysis Outcome --------------------
Accurate Location: {'entity': 'United States', 'lat': 37.09024, 'lon': -95.712891}
Projected Primary Energy Consumption per capita (kWh/person): 75652.63771000017
Energy Mix: {'Fossil': 2745.3333333333335, 'Renewables': 487.33809523809515, 'Nuclear': 790.6004761904763}

In the target year (3020), the average person in United States is predicted to consume about 75652.64kWh of energy, as predicted by the rf model.



To view the full prediction, type 'FP' (case-sensitive):  


In [63]:
print("Available Input Options:\n")

df = pl.read_csv("data.csv", infer_schema_length=10000, try_parse_dates=True)
print("Available Entities:", set(df["Entity"]))

print("\nAvailable Models: 'linear', 'poly', and 'rf'")

print("\nAvailable Years: 2000 to 2020 (existing), 2020 onwards (predictive)")

Available Input:

Available Entities: {'Cayman Islands', 'Ghana', 'Eswatini', 'United States', 'Honduras', 'Mauritania', 'New Zealand', 'Puerto Rico', 'Uzbekistan', 'Kuwait', 'Malawi', 'Bangladesh', 'Iraq', 'Myanmar', 'Zambia', 'Gambia', 'Costa Rica', 'Djibouti', 'Namibia', 'Sao Tome and Principe', 'Belgium', 'Iceland', 'Sudan', 'Japan', 'Antigua and Barbuda', 'Bermuda', 'Denmark', 'Guinea', 'Botswana', 'Libya', 'Nepal', 'North Macedonia', 'Spain', 'Chad', 'Ireland', 'Madagascar', 'Saudi Arabia', 'Jamaica', 'Germany', 'Central African Republic', 'United Kingdom', 'Ecuador', 'Kiribati', 'Uganda', 'Mongolia', 'Hungary', 'Lebanon', 'Samoa', 'Indonesia', 'Equatorial Guinea', 'Chile', 'South Sudan', 'Zimbabwe', 'France', 'Israel', 'Jordan', 'Guinea-Bissau', 'Ukraine', 'Australia', 'Belize', 'Togo', 'Peru', 'Bulgaria', 'Czechia', 'El Salvador', 'Eritrea', 'Mauritius', 'Tunisia', 'India', 'Colombia', 'Cyprus', 'Burkina Faso', 'Yemen', 'Canada', 'Lithuania', 'Saint Lucia', 'China', 'Egypt', 'K